# Develop a random forest model for RDKit descriptors and compare results.

In [1]:
!pip install scikit-learn
!pip install rdkit-pypi

In [2]:
# import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

# read BBBP.csv file
df = pd.read_csv('BBBP.csv')
df.head()

,num,name,p_np,smiles
0,1,Propanolol,1,[Cl].CC(C)NCC(O)COc1cccc2ccccc12
1,2,Terbutylchlorambucil,1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl
2,3,40730,1,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...
3,4,24,1,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C
4,5,cloxacillin,1,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...


In [5]:
# split training, validation, and test set into 8:1:1 ratio
from sklearn.model_selection import train_test_split
train, temp = train_test_split(df, test_size=0.2, random_state=42)
test, val = train_test_split(temp, test_size=0.5, random_state=42)

# verify length of each set
print("Training set length: ", len(train))
print("Validation set length: ", len(val))
print("Test set length: ", len(test))

Training set length:  1640
Validation set length:  205
Test set length:  205


In [6]:
# check for duplicates and remove them
print("Number of duplicates in training set: ", train.duplicated().sum())

Number of duplicates in training set:  0


In [24]:
# calculate molecular descriptors for each molecule from BBBP.csv
# append both lists if mol != None
desc_list = []
p_np = []
for i in range(len(df)):
    mol = Chem.MolFromSmiles(df['smiles'][i])
    if mol != None:
        desc_list.append(Descriptors.MolWt(mol))
        p_np.append(df['p_np'][i])

[21:10:06] Explicit valence for atom # 1 N, 4, is greater than permitted
[21:10:06] WARNING: not removing hydrogen atom without neighbors
[21:10:06] Explicit valence for atom # 6 N, 4, is greater than permitted
[21:10:06] WARNING: not removing hydrogen atom without neighbors
[21:10:06] WARNING: not removing hydrogen atom without neighbors
[21:10:06] WARNING: not removing hydrogen atom without neighbors
[21:10:06] WARNING: not removing hydrogen atom without neighbors
[21:10:06] WARNING: not removing hydrogen atom without neighbors
[21:10:06] WARNING: not removing hydrogen atom without neighbors
[21:10:06] Explicit valence for atom # 6 N, 4, is greater than permitted
[21:10:06] WARNING: not removing hydrogen atom without neighbors
[21:10:06] WARNING: not removing hydrogen atom without neighbors
[21:10:06] WARNING: not removing hydrogen atom without neighbors
[21:10:06] WARNING: not removing hydrogen atom without neighbors
[21:10:06] Explicit valence for atom # 11 N, 4, is greater than pe

In [25]:
# convert lists to numpy arrays
desc_list = np.array(desc_list)
p_np = np.array(p_np)

In [26]:
# print length of each array
print("Length of molecular descriptors: ", len(desc_list))
print("Length of p_np: ", len(p_np))

Length of molecular descriptors:  2039
Length of p_np:  2039


In [29]:
# list default hyperparameters: n_estimators, max_depth, min_samples_leaf, min_impurity_decrease, max_features
# n_estimators: number of trees in the forest
# max_depth: maximum depth of the tree
# min_samples_leaf: minimum number of samples required to be at a leaf node
# min_impurity_decrease: minimum impurity decrease required to split a node
# max_features: number of features to consider when looking for the best split

# train random forest model with default hyperparameters
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42)
rf.fit(desc_list.reshape(-1,1), p_np)

# predict p_np values for validation set
pred = rf.predict(val['smiles'].apply(Chem.MolFromSmiles).apply(Descriptors.MolWt).values.reshape(-1,1))

# calculate accuracy
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(val['p_np'], pred))

Accuracy:  0.9658536585365853


[21:11:23] WARNING: not removing hydrogen atom without neighbors
[21:11:23] WARNING: not removing hydrogen atom without neighbors
[21:11:23] WARNING: not removing hydrogen atom without neighbors
[21:11:23] WARNING: not removing hydrogen atom without neighbors
[21:11:23] WARNING: not removing hydrogen atom without neighbors
[21:11:23] WARNING: not removing hydrogen atom without neighbors
[21:11:23] WARNING: not removing hydrogen atom without neighbors


In [30]:
# optimize hyperparameters using grid search
from sklearn.model_selection import GridSearchCV

# define hyperparameters
param_grid = {'n_estimators': [100, 200, 300, 400, 500],
                'max_depth': [2, 4, 6, 8, 10],
                'min_samples_leaf': [1, 2, 4],
                'min_impurity_decrease': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
                'max_features': ['auto', 'sqrt', 'log2']}

# define grid search
grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')

# fit grid search
grid.fit(desc_list.reshape(-1,1), p_np)

# print best hyperparameters
print("Best n_estimators: ", grid.best_estimator_.n_estimators)
print("Best max_depth: ", grid.best_estimator_.max_depth)
print("Best min_samples_leaf: ", grid.best_estimator_.min_samples_leaf)
print("Best min_impurity_decrease: ", grid.best_estimator_.min_impurity_decrease)
print("Best max_features: ", grid.best_estimator_.max_features)

/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

Best n_estimators:  400
Best max_depth:  8
Best min_samples_leaf:  4
Best min_impurity_decrease:  0.0
Best max_features:  auto


In [32]:
# use best hyperparameters to train random forest model
rf = RandomForestClassifier(n_estimators=grid.best_estimator_.n_estimators, max_depth=grid.best_estimator_.max_depth, min_samples_leaf=grid.best_estimator_.min_samples_leaf, min_impurity_decrease=grid.best_estimator_.min_impurity_decrease, max_features=grid.best_estimator_.max_features, random_state=42)
rf.fit(desc_list.reshape(-1,1), p_np)


/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomForestClassifier(max_depth=8, max_features='auto', min_samples_leaf=4,
                       n_estimators=400, random_state=42)

In [33]:
# predict p_np values for validation set
pred = rf.predict(val['smiles'].apply(Chem.MolFromSmiles).apply(Descriptors.MolWt).values.reshape(-1,1))

# calculate accuracy
print("Accuracy: ", accuracy_score(val['p_np'], pred))

Accuracy:  0.824390243902439


[22:04:12] WARNING: not removing hydrogen atom without neighbors
[22:04:12] WARNING: not removing hydrogen atom without neighbors
[22:04:12] WARNING: not removing hydrogen atom without neighbors
[22:04:12] WARNING: not removing hydrogen atom without neighbors
[22:04:12] WARNING: not removing hydrogen atom without neighbors
[22:04:12] WARNING: not removing hydrogen atom without neighbors
[22:04:12] WARNING: not removing hydrogen atom without neighbors


# Compared with using morgan fingerprints, calculating molecular descriptors yields better accuracy. However, compared with the literature value, the value is still lower than what the literature reported. This, again, could be due to less parameters being trained compared with the literature.